## Decorators 101
A decorator is a callable that takes another function as argument (the decorated func‐ tion) The decorator may perform some processing with the decorated function, and returns it or replaces it with another function or callable object.

In [5]:
# Example 7-1: A decorator usually replaces a function with a different one
def deco(func):
    def inner():
        print('running inner()')
    return inner

# target = deco(target)
@deco
def target():
    print('running target()')

In [6]:
target()

running inner()


In [7]:
target

<function __main__.deco.<locals>.inner()>

## Why Python Executes Decorators
A key feature of decorators is that they run right after the decorated function is defined. That is usually at import time (i.e., when a module is loaded by Python). Consider registration.py in Example 7-2.

In [10]:
# Example 7-2: The registration module
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running register(<function f1 at 0x106275048>)
running register(<function f2 at 0x106311d08>)


In [11]:
registry

[<function __main__.f1()>, <function __main__.f2()>]

In [12]:
f1()

running f1()


In [13]:
f2()

running f2()


In [14]:
f3()

running f3()


## Decorator-Enhanced Strategy Pattern

In [15]:
# Exmaple 7-3: The promos list is filled by the promotion decorator
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    '''5% discount for customers with 1000 or more fidelity points'''
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    '''10% discount for each LineItem with 20 or more units'''
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    '''7% discount for orders with 10 or more distinct items'''
    dictinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    '''Select best discount available'''
    return max(promo(order) for promo in promos)

## Variable Scope Rules
In Example 7-4, we define and test a function that reads two variables: a local variable a, defined as function parameter, and variable b that is not defined anywhere in the function.

In [16]:
# Example 7-4: Function reading a local and global variable
def f1(a):
    print(a)
    print(b)

In [17]:
f1(3)

3


NameError: name 'b' is not defined

In [18]:
b = 6
f1(3)

3
6


In [21]:
# Example 7-5. Variable b is local, because it is assigned a value in the body of the function
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

In [22]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [23]:
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

In [25]:
f3(3)

3
6


## Cloures
Actually, a closure is a function with an extended scope that encompasses nonglobal variables referenced in the body of the function but not defined there. It does not matter whether the function is anonymous or not; what matters is that it can access nonglobal variables that are defined outside of its body.

In [31]:
# Example 7-8: A class to calculate a running average.
class Averager():
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [32]:
avg = Averager()

In [34]:
avg(10)

10.0

In [35]:
avg(11)

10.5

In [36]:
avg(12)

11.0

In [37]:
# Example 7-9: A higher-order function to calculate a running average
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

In [38]:
avg = make_averager()

In [39]:
avg(10)

10.0

In [40]:
avg(11)

10.5

In [41]:
avg(12)

11.0

### Description of example 7-9
Note that series is a local variable of make_averager because the initialization series = [] happens in the body of that function. But when avg(10) is called, make_averager has already returned, and its local scope is long gone.
Within averager, series is a free variable. This is a technical term meaning a variable that is not bound in the local scope.

The closure for averager extends the scope of that function to include the binding for the free variable series.

Inspecting the returned averager object shows how Python keeps the names of local and free variables in the __code__ attribute that represents the compiled body of the function. Example 7-11 demonstrates.

In [43]:
# Example 7-11: Inspecting the function created by make_average.
avg.__code__.co_varnames

('new_value', 'total')

In [44]:
avg.__code__.co_freevars

('series',)

The binding for series is kept in the \__closure__ attribute of the returned function avg. Each item in avg.\__closure__ corresponds to a name in avg.\__code__.co_free vars. These items are cells, and they have an attribute called cell_contents where the actual value can be found. 

In [45]:
avg.__closure__

(<cell at 0x1062a5678: list object at 0x106163708>,)

In [46]:
avg.__closure__[0].cell_contents

[10, 11, 12]

To summarize: a closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.

## The nonlocal Declaration

In [3]:
# Example 7-13: A broken higher-order function to calculate a running average without keeping all history
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        # The problem is: below statement count += 1 actually means count = count + 1, when count is immutable type
        count += 1
        total += new_value
        return total / count
    return averager

In [7]:
avg = make_averager()

In [8]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [16]:
# Example 7-13: A broken higher-order function to calculate a running average without keeping all history
def make_averager():
    count = [0]
    total = [0]
    
    def averager(new_value):
        # 即使count和total是可变类型，也人仍然会有上述问题，原因是count在执行count += [1]的时候，由于该表达式是一个赋值操作，
        # 所以会在该函数作用域中创建一个count变量，然而它是自增操作，count同时被引用和被赋值，故报异常，
        # 暂时没发现与可变或不可变类型有更深入的关系
        count += [1]
        total += [new_value]
        return total / count
    return averager

In [17]:
avg = make_averager()

In [18]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

We did not have this problem in Example 7-9 because we never assigned to the ser ies name; we only called series.append and invoked sum and len on it. So we took advantage of the fact that lists are mutable.

But with immutable types like numbers, strings, tuples, etc., all you can do is read, but never update. If you try to rebind them, as in count = count + 1, then you are implicitly creating a local variable count. It is no longer a free variable, and therefore it is not saved in the closure.

To work around this, the nonlocal declaration was introduced in Python 3. It lets you flag a variable as a free variable even when it is assigned a new value within the function. If a new value is assigned to a nonlocal variable, the binding stored in the closure is changed. A correct implementation of our newest make_averager looks like Example 7-14.

In [56]:
# Example 7-14: Calculate a running average without keeping all history(fixed with the use of nonlocal)
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

In [57]:
avg = make_averager()
avg(10)

10.0